# Top Routes

This notebook calculates and exports bar charts of:

- Top routes (unidirectional) over all time
- Top routes (bidirectional) over all time

In the unidirectional chart, LHR-JFK and JFK-LHR are distinct whereas in the bidirectional chart they are the same route.

To use it, set the top 'N' value and export format for charts in the first code cell before running the notebook.

In [ ]:
# Top "N" routes to include
top_n = 20

# Export format for the charts:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
from pathlib import Path
import sqlparse

# Read the query file
query_file_path = Path("sql") / "sightings.sql"
with open(query_file_path.absolute(), "r") as f:
    query = f.read().replace("\n", " ")

# Show a pretty-printed form of the query
print(sqlparse.format(query, reindent=True, keyword_case="upper"))

In [ ]:
import pandas as pd
import sqlite3
import os

# Connect to the database, execute the query and read the results into a dataframe
database_path = os.environ["FLIGHT_RECORDER_DB"]
connection = sqlite3.connect(database_path)
df = pd.read_sql_query(query, connection, parse_dates=["Date"])

# Check there is some data
if not df.shape[0]:
    message = f"No data found"
    raise ValueError(message)

# Preview the data
df.head()

In [ ]:
import pandas as pd
import warnings

# Remove entries where either the point of embarkation or destination aren"t available
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
df_cleaned = df[(df["Embarkation"] != "N/A") & (df["Destination"] != "N/A")]

# Combine embarkation and destination into a single route column
df_cleaned["Route"] = df_cleaned["Embarkation"].str.strip() + "-" + df_cleaned["Destination"].str.strip()

# Count unidirectional routes. In this scheme, LHR-JFK is different from JFK-LHR
route_counts = df_cleaned["Route"].value_counts().reset_index()
route_counts.columns = ["Route", "Sightings"]

# Define a function to normalise a route
def normalize_route(row):
    airports = sorted([row["Embarkation"].strip(), row["Destination"].strip()])
    return f"{airports[0]}-{airports[1]}"

# Use it to add a normalised route column to the data
df_cleaned["Normalised_Route"] = df_cleaned.apply(normalize_route, axis=1)

# Count normalized route frequencies
normalised_counts = df_cleaned["Normalised_Route"].value_counts().reset_index()
normalised_counts.columns = ["Normalised_Route", "Sightings"]

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Create a Pandas Excel writer
output_path = export_folder_path / f"Top-Routes.xlsx"

with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
    # Daily data
    route_counts.to_excel(writer, sheet_name="Unidirectional", index=False)
    
    # Monthly data
    normalised_counts.to_excel(writer, sheet_name="Bidirectional", index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Generate the unidirectional bar chart
plt.figure(figsize=(12, 6))
sns.barplot(data=route_counts.head(top_n), x="Sightings", y="Route", palette="Blues_r", hue="Route")
plt.title(f"Top {top_n} Most Common Aircraft Routes (Unidirectional)")
plt.xlabel("Number of Sightings")
plt.ylabel("Route")
plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"Top-Routes-Unidirectional.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"Top-Routes-Unidirectional.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the chart
plt.show()


In [ ]:
# Generate the bidirectional bar chart
plt.figure(figsize=(12, 6))
sns.barplot(data=normalised_counts.head(top_n), x="Sightings", y="Normalised_Route", palette="magma", hue="Normalised_Route")
plt.title(f"Top {top_n} Most Common Aircraft Routes (Bidirectional)")
plt.xlabel("Number of Sightings")
plt.ylabel("Route")
plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"Top-Routes-Bidirectional.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"Top-Routes-Bidirectional.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the chart
plt.show()